In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_rop',
    'store_status',
    proc_date,
    'temp_rop_store_status'
)

In [0]:
spark.sql(f""" create table if not exists {catalog_name}.udp_wcm_silver_rop.store_status like {catalog_name}.udp_wcm_bronze_rop.store_status""")

In [0]:
spark.sql(f"""ALTER TABLE {catalog_name}.udp_wcm_silver_rop.store_status
SET TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_rop.store_status
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
        INSERT INTO {catalog_name}.udp_wcm_silver_rop.store_status 
        SELECT store_id, store_name, validate_from, validate_to, reason, comment, 
           updated_date, updated_user, proc_date,file_creation_ts
        FROM temp_rop_store_status
        WHERE proc_date = DATE('{proc_date}')
    """)